# Model builder
## Life Cycle Assessment of ``{Operational Energy Moerschberg for 1 year: 1}``
###[CONTEXT]
The model relies on datapackages to ensure replicability of the calculation.


In [212]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing as bwp
import numpy as np
from pathlib import Path
import pandas as pd


In [213]:
bd.projects.delete_project('ei38-teaching-25')

'default'

In [214]:
bi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")
bd.projects.set_current('ei38-teaching-25')


Restoring project backup archive - this could take a few minutes...


In [215]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei 3.8 cutoff

In [216]:
# bd.projects.migrate_project_25()

In [217]:
if "foreground" in bd.databases:
    del bd.databases["foreground"]
    
foreground_importer = bi.ExcelImporter("./data/inputs/lci_moeschberg.xlsx")
foreground_importer.apply_strategies()
foreground_importer.match_database("biosphere3", fields=("name", "unit", "categories"))
foreground_importer.match_database("ei 3.8 cutoff", fields=("name", "unit", "location"))
foreground_importer.statistics()
foreground_importer.write_database()

Extracted 1 worksheets in 0.00 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.83 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
5 datasets
18 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 da

In [218]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ei 3.8 cutoff
	energy_moeschberg

In [219]:
fu = bd.get_activity(database="energy_moeschberg", name = "energy demand, operational, Hotel Moeschberg")
fu

'energy demand, operational, Hotel Moeschberg' (unit, CH, None)

In [220]:
heat = bd.get_activity(database="energy_moeschberg", name = "heat supply, Hotel Moeschberg, 2021")

In [221]:
wood_pellets_heat = bd.get_node(database="ei 3.8 cutoff", name='wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future', unit='megajoule')

In [222]:
wood_pellets_market = bd.get_node(database="ei 3.8 cutoff", name = "market for wood pellet, measured as dry mass", location = "RER")

In [223]:
wood_pellets_prod_RER = bd.get_node(database="ei 3.8 cutoff", name = "wood pellet production", location = "RER")

In [224]:
if "Regionalized_db" in bd.databases:
    del bd.databases["Regionalized_db"]

In [225]:
Regionalized_db = bd.Database("Regionalized_db", backend = "iotable") #need iotable to be able to do the regionalization

In [226]:
Regionalized_db.write({})

In [227]:
bd.databases

Databases dictionary with 4 object(s):
	Regionalized_db
	biosphere3
	ei 3.8 cutoff
	energy_moeschberg

In [228]:
foreground=bd.Database('energy_moeschberg')

In [229]:
regionalized_fu=Regionalized_db.new_node(code="new-fu", name="energy demand, operational, Hotel Moeschberg - regionalized", location="CH", unit="unit")
regionalized_fu.save()

In [230]:
regionalized_heat=Regionalized_db.new_node(code="new-heat", name='heat supply, Hotel Moeschberg, 2021 - regionalized', location="CH", unit="megajoule")
regionalized_heat.save()

In [231]:
new_wood_pellets_heat = Regionalized_db.new_node(code="pellets-heat", name="wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future - regionalized", location="CH", unit="megajoule")
new_wood_pellets_heat.save()

In [232]:
new_wood_pellets_market = Regionalized_db.new_node(code="pellets-market", name="market for wood pellet, measured as dry mass", location="CH", unit="kilogram")
new_wood_pellets_market.save()

In [233]:
new_pellets_prod = Regionalized_db.new_node(code="pellets-prod", name="wood pellet production", location="CH", unit="kilogram")
new_pellets_prod.save()

In [234]:
from collections import defaultdict

new_pellets_tech = defaultdict(float)

for exc in wood_pellets_prod_RER.technosphere():
    new_pellets_tech[bd.get_node(
        database = "ei 3.8 cutoff",
        name = exc.input['name'],
        location = exc.input['location']
    )] += exc['amount']

In [235]:
tech_edges = [
    {"row": regionalized_fu.id, "col": regionalized_fu.id, "amount":1},
    {"row": fu.id, "col": regionalized_fu.id, "amount":1, "flip":True}, #consume the old activity to copy exchanges
    {"row": heat.id, "col": regionalized_fu.id, "amount":655489 , "flip":False},
    {"row": regionalized_heat.id, "col": regionalized_fu.id, "amount":655489, "flip":True},
] + [
    {"row": regionalized_heat.id, "col": regionalized_heat.id, "amount":1},
    {"row": heat.id, "col": regionalized_heat.id, "amount":1, "flip":True}, #consume the old activity to copy exchanges
    {"row": wood_pellets_heat.id, "col": regionalized_heat.id, "amount":0.9953, "flip":False},
    {"row": new_wood_pellets_heat.id, "col": regionalized_heat.id, "amount":0.9953, "flip":True},
] + [
    {"row": new_wood_pellets_heat.id, "col": new_wood_pellets_heat.id, "amount":1},
    {"row": wood_pellets_heat.id, "col": new_wood_pellets_heat.id, "amount":1, "flip":True}, #consume the old activity to copy exchanges
    {"row": new_wood_pellets_market.id, "col": new_wood_pellets_heat.id, "amount":0.0229911254795743, "flip":True},
    {"row": wood_pellets_market.id, "col": new_wood_pellets_heat.id, "amount": 0.0229911254795743},
] + [
    {"row": new_wood_pellets_market.id, "col": new_wood_pellets_market.id, "amount":1},
    {"row": wood_pellets_market.id, "col": new_wood_pellets_market.id, "amount":1, "flip":True},
    {"row": new_pellets_prod.id, "col": new_wood_pellets_market.id, "amount": 1, "flip":True},
    {"row": wood_pellets_prod_RER.id, "col": new_wood_pellets_market.id, "amount": 1},
] + [
    {"row": new_pellets_prod.id, "col": new_pellets_prod.id, "amount": 1},
    {"row": wood_pellets_prod_RER.id, "col": new_pellets_prod.id, "amount": 1, "flip":True},
    {
        "row": bd.get_activity(
            database="ei 3.8 cutoff", 
            name='market group for electricity, medium voltage',
            location='RER'
        ).id, 
        "col": new_pellets_prod.id, 
        "amount": 0.096
    },
] + [
    {"row": key.id, "col": new_pellets_prod.id, "amount": value}
    for key, value in new_pellets_tech.items() if key['location']=="Europe without Switzerland"
] + [
    {
        "row": bd.get_activity(
            database="ei 3.8 cutoff", 
            name='market for electricity, medium voltage',
            location='CH'
        ).id, 
        "col": new_pellets_prod.id, 
        "amount": 0.096,
        "flip":True
    },
] + [
    {"row": bd.get_activity(database="ei 3.8 cutoff", name=key['name'], location="CH").id, 
      "col": new_pellets_prod.id, "amount": value, "flip":True}
    for key, value in new_pellets_tech.items() if key['location']=="Europe without Switzerland"

]

In [236]:
Regionalized_db.write_exchanges(tech_edges, [], ["ei 3.8 cutoff"])

Starting IO table write
Adding technosphere matrix
Adding biosphere matrix
Finalizing serialization


In [237]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [238]:
fu.new_edge(input=regionalized_fu, amount=0, type="technosphere").save()

In [239]:
_, data_objs, _ = bd.prepare_lca_inputs({fu: 1}, ipcc)

In [240]:
lca = bc.LCA({fu.id: 1}, data_objs=data_objs)
lca.lci()
lca.lcia()
lca.score

4288.388560965337

In [242]:
lca.lcia({regionalized_fu.id: 1})
lca.score

3629.403444136348

In [243]:
imp = bi.ExcelImporter("./data/inputs/lci_solarpv_huimin_masterthesis_combined.xlsx")
imp.apply_strategies()
imp.match_database("ei 3.8 cutoff", fields=('name','unit','location'))
#imp.match_database("ecoinvent_remind_SSP2-Base_2020", fields=('name','unit','location','reference product'))
imp.match_database(fields=('name', 'unit', 'location'))
imp.statistics()
imp.write_excel(only_unlinked=True)
list(imp.unlinked)

imp.write_database()


Extracted 1 worksheets in 0.03 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.98 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
18 datasets
386 exchanges
0 unlinked exchanges
  
Wrote matching file to:
/home/jupyter

In [244]:
bd.databases

Databases dictionary with 5 object(s):
	Regionalized_db
	biosphere3
	ei 3.8 cutoff
	energy_moeschberg
	solar_huimin

In [245]:
solar=bd.Database('solar_huimin')

In [247]:
PV_new=[act for act in solar if 'Electricity production, 3kWp multi-Si' in act['name']][0]
PV_new

'Electricity production, 3kWp multi-Si photovoltaic panel slanted-roof installation' (kilowatt hour, CN, None)